In [ ]:
import sys
import os
sys.path.append("../..")

In [ ]:
# import libraries and custom modules
import sklearn
sklearn.set_config(transform_output="pandas")
import importlib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from rapidfuzz import process, fuzz
from geopy.distance import geodesic
from category_encoders.target_encoder import TargetEncoder


import survival.utils
import survival.constants
importlib.reload(survival.utils)
importlib.reload(survival.constants)
from survival.utils import show_all
from survival.utils import validate_imputation



In [ ]:
# load data
data = pd.read_parquet("../../data/processed/raw_clean.parquet")
hh = pd.read_csv("../../data/processed/hh_clean.csv")
cities = pd.read_csv('../../data/processed/cities.csv')
nl = pd.read_csv('../../data/processed/nl.csv')
countries = pd.read_csv('../../data/processed/countries.csv')
income = pd.read_csv('../../data/processed/income.csv')
gemeenten = pd.read_csv('../../data/processed/gemeenten.csv')
postalcode = pd.read_csv('../../data/processed/postalcode.csv')
reg = pd.read_csv('../../data/processed/operaballet_reg_prods_clean.csv')

In [ ]:
retrieval_date = pd.Timestamp('2025-05-12')

In [ ]:
min_purchase_dates_agg = data.groupby(['id', 'start_date'], observed=True)['purchase_date'].transform('min')

data['min_purchase_date'] = min_purchase_dates_agg

In [ ]:
# per id per start_date, count the amount of tickets bought and store in column 'order_size'
# count per ticket type the amount of tickets bought and store in columns 'order_size_<ticket_type>'. 
# fill with 0 if no tickets bought
data['order_size'] = data.groupby(['id', 'start_date'])['id'].transform('count')
data['total_order_value'] = data.groupby(['id', 'start_date'])['price'].transform('sum')
data['avg_order_value'] = data.groupby(['id', 'start_date'])['price'].transform('mean')
data['total_order_value'] = data['total_order_value'].round(2)
data['avg_order_value'] = data['avg_order_value'].round(2)

In [ ]:
# drop these
# but perhaps drop ticket_num earlier -> figure out if its necessary in a grouping operation
data = data.drop(columns=['price'])

In [ ]:
daily_sales = data.groupby(['start_date', 'production', 'min_purchase_date']).size().reset_index(name='tickets_sold')
    
# cumulative sales for each performance
perf_sales_cumulative = daily_sales.sort_values(['start_date', 'min_purchase_date'])
perf_sales_cumulative['perf_sales_cumulative'] = perf_sales_cumulative.groupby('start_date')['tickets_sold'].cumsum()

# cumulative sales for each production
prod_sales_cumulative = daily_sales.sort_values(['production', 'min_purchase_date'])
prod_sales_cumulative['prod_sales_cumulative'] = prod_sales_cumulative.groupby('production')['tickets_sold'].cumsum()

# daily sales for each performance 
perf_sales_daily = daily_sales.groupby(['start_date', 'min_purchase_date'])['tickets_sold'].sum().reset_index(name='daily_perf_sales')

# daily sales for each production 
prod_sales_daily = daily_sales.groupby(['production', 'min_purchase_date'])['tickets_sold'].sum().reset_index(name='daily_prod_sales')

perf_to_merge = perf_sales_cumulative[['start_date', 'min_purchase_date', 'perf_sales_cumulative']].drop_duplicates(
    subset=['start_date', 'min_purchase_date'], keep='last'
)
prod_to_merge = prod_sales_cumulative[['production', 'min_purchase_date', 'prod_sales_cumulative']].drop_duplicates(
    subset=['production', 'min_purchase_date'], keep='last'
)

# final sales totals for each performance
final_perf_sales = daily_sales.groupby('start_date')['tickets_sold'].sum().reset_index(name='final_perf_sales')

data = data.merge(
    perf_to_merge,
    on=['start_date', 'min_purchase_date'],
    how='left'
)
data = data.merge(
    prod_to_merge,
    on=['production', 'min_purchase_date'],
    how='left'
)
data = data.merge(
    perf_sales_daily[['start_date', 'min_purchase_date', 'daily_perf_sales']],
    on=['start_date', 'min_purchase_date'],
    how='left'
)
data = data.merge(
    prod_sales_daily[['production', 'min_purchase_date', 'daily_prod_sales']],
    on=['production', 'min_purchase_date'],
    how='left'
)

data = data.merge(
    final_perf_sales, 
    on='start_date', 
    how='left'
    )

# sales percentage (how much of final sales occurred by purchase date)
data['perf_sales_percentage'] = data['perf_sales_cumulative'] / data['final_perf_sales']

In [ ]:
reg['sale_start_date'] = pd.to_datetime(reg['sale_start_date'], errors='coerce')

# merge sale start dates from reg to main data
reg_lookup = reg[['production', 'sale_start_date']].drop_duplicates(subset=['production'])

# if sale_start_date already exists in data
has_sale_start_date = 'sale_start_date' in data.columns

# merge with appropriate suffix 
if has_sale_start_date:
    data = pd.merge(data, reg_lookup, on='production', how='left', suffixes=('', '_reg'))
    # if column exists after merge, use to fill nans in the main column
    if 'sale_start_date_reg' in data.columns:
        data['sale_start_date'] = data['sale_start_date'].fillna(data['sale_start_date_reg'])
        data = data.drop(columns=['sale_start_date_reg'])
else:
    data = pd.merge(data, reg_lookup, on='production', how='left')

# productions with unknown start dates, find earliest purchase date
missing_start_prods = [
    '22/23 messa da requiem flirt', '22/23 forsythe flirt',
    '23/24 die zauberflote flirt', '23/24 frida flirt', '23/24 gala hnb',
    '23/24 dansers van morgen', '24/25 don quichot flirt'
]

# get earliest purchase date for each production needing imputation
mask = data['production'].isin(missing_start_prods) & data['sale_start_date'].isna()
if mask.any():
    # group by production and get min purchase date
    imputed_dates = data[mask].groupby('production')['min_purchase_date'].min()
    
    # apply imputed dates where needed
    for prod, min_date in imputed_dates.items():
        data.loc[(data['production'] == prod) & data['sale_start_date'].isna(), 'sale_start_date'] = min_date

# date columns to datetime
data['min_purchase_date'] = pd.to_datetime(data['min_purchase_date'], errors='coerce')
data['sale_start_date'] = pd.to_datetime(data['sale_start_date'], errors='coerce') 
data['start_date'] = pd.to_datetime(data['start_date'], errors='coerce')

# create features
data['days_since_sale_start'] = (data['min_purchase_date'] - data['sale_start_date']).dt.days
data['sale_period_duration_days'] = (data['start_date'] - data['sale_start_date']).dt.days

# remove negative days since sale start
data = data[data['days_since_sale_start'] >= 0]

In [ ]:
# find the last performance date for each season
season_end_dates = data.groupby('season')['start_date'].max().reset_index()
season_end_dates.rename(columns={'start_date': 'season_last_performance'}, inplace=True)

# merge back to main data
data = data.merge(season_end_dates, on='season', how='left')

# days left in season from performance date to last performance
data['days_left_in_season'] = (data['season_last_performance'] - data['start_date']).dt.days

In [ ]:
# subscription tickets
subscription_ticket = ['abo standaard', 'abo vk dno', 'abonnement 22/23', 'kassa abo standaard',
                       'abonnement 24/25', 'abo vk hnb', 'abo vrij', 'serie 2025-2026']

# subscriber tickets
data['is_subscriber'] = data['ticket_type'].isin(subscription_ticket).astype(int)

# subscriber tickets per performance
subscriber_count = data[data['is_subscriber'] == 1].groupby('start_date')['order_size'].sum().reset_index(name='subscriber_tickets')

# merge subscriber count
data = data.merge(subscriber_count, on='start_date', how='left')

# nan values (performances with no subscribers)
data['subscriber_tickets'] = data['subscriber_tickets'].fillna(0)

# percentages
data['subscriber_percentage'] = data['subscriber_tickets'] / 1633 
data['regular_tickets'] = data['final_perf_sales'] - data['subscriber_tickets']
data['regular_percentage'] = data['regular_tickets'] / 1633
data['occupancy_rate'] = data['final_perf_sales'] / 1633

# remove all subscription tickets from activity
data = data[~data['ticket_type'].isin(subscription_ticket)]

In [ ]:
# free tickets
free_tickets = data[data['is_free'] == 1].groupby(['start_date', 'min_purchase_date']).size().reset_index(name='free_tickets')

# Cumulative free tickets for each performance
perf_free_tickets_cumulative = free_tickets.sort_values(['start_date', 'min_purchase_date'])
perf_free_tickets_cumulative['perf_free_tickets_cumulative'] = perf_free_tickets_cumulative.groupby('start_date')['free_tickets'].cumsum()

# operational discounts
operational_discount = data.loc[data['ticket_type'].isin(['huiskorting', 'medewerker no&b', 'tell a friend']), 
                           ['start_date', 'min_purchase_date']].groupby(['start_date', 'min_purchase_date']).size().reset_index(name='operational_discount')

# Cumulative company discount for each performance
perf_operational_discount_cumulative = operational_discount.sort_values(['start_date', 'min_purchase_date'])
perf_operational_discount_cumulative['perf_operational_discount_cumulative'] = perf_operational_discount_cumulative.groupby('start_date')['operational_discount'].cumsum()

# Merge free tickets data
data = data.merge(
    perf_free_tickets_cumulative[['start_date', 'min_purchase_date', 'perf_free_tickets_cumulative']],
    on=['start_date', 'min_purchase_date'],
    how='left'
)

# Merge company discount data
data = data.merge(
    perf_operational_discount_cumulative[['start_date', 'min_purchase_date', 'perf_operational_discount_cumulative']],
    on=['start_date', 'min_purchase_date'],
    how='left'
)

# Fill missing values with 0
data['perf_free_tickets_cumulative'] = data['perf_free_tickets_cumulative'].fillna(0)
data['perf_operational_discount_cumulative'] = data['perf_operational_discount_cumulative'].fillna(0)

In [ ]:
# delete records where is_free == 1 and drop the column
data = data[data['is_free'] != 1]
data = data.drop('is_free', axis=1)
data = data.reset_index(drop=True)

In [ ]:
# drop all educatie tickets because these visitors are not unique
data = data[~data['ticket_type'].str.contains('educatie')]

# drop all ticket where ticket_type are related to employees
irregular_ticket = [
    'zoekplaats',
    'huiskorting',
    'medewerker',
    'medewerker no&b',
    'vrijplaats',
    'paniek',
    'balletorkest',
    'orkest',
    'nedpho',
    'ckv leerling',
    'ckv docent',
    'streaming mia',
    'groep -5%',
    'groep -10%'
    ]

data = data[~data['ticket_type'].isin(irregular_ticket)]

# drop all records where email contains @operaballet.nl
data = data[data['email'].notna() & ~data['email'].str.contains('@operaballet.nl', na=False)]

# drop the following ids because they are related to employees, institutions or groups
from survival.constants import nonvisitor_ids
data = data[~data['id'].isin(nonvisitor_ids)]

data = data.reset_index(drop=True)



In [ ]:
# consolidate ticket type names using ticket_categorie_mapping_dict
from survival.constants import ticket_category_mapping_dict
# flatten the values in ticket_category_mapping_dict
flattened_ticket_categories = {item for sublist in ticket_category_mapping_dict.values() for item in sublist}
# which values are in data[ticket_type].unique() that are not in the ticket_category_mapping_dict values
missing_ticket_types = set(data['ticket_type'].unique()) - flattened_ticket_categories
missing_ticket_types

In [ ]:
# if a ticket_type name matches a value in the dict, replace it with the key
# map ticket types to categories
data['ticket_type'] = data['ticket_type'].map({ticket: category for category, tickets in ticket_category_mapping_dict.items() for ticket in tickets}).fillna('other')


In [ ]:
# rank dictionaries
ballet_rank_replace_dict = {
    'premium': 1, 'rang 1': 2, 'rang 2': 3, 'rang 3': 4,
    'rang 4': 5, 'rang 5': 6, 'rang 6': 7
}
opera_rank_replace_dict = {
    'rang 1': 1, 'rang 2': 2, 'rang 3': 3, 'rang 4': 4,
    'rang 5': 5, 'rang 6': 6, 'rang 7': 7
}

# dummies for artform
data = pd.get_dummies(data, columns=['artform'], prefix='artform', drop_first=False)
rename_dict = {}
if 'artform_ballet' in data.columns: rename_dict['artform_ballet'] = 'ballet'
if 'artform_opera' in data.columns: rename_dict['artform_opera'] = 'opera'
if rename_dict: data = data.rename(columns=rename_dict)
if 'ballet' in data.columns: data['ballet'] = data['ballet'].astype(int)
if 'opera' in data.columns: data['opera'] = data['opera'].astype(int)


# numeric_rank column with nan
data['numeric_rank'] = np.nan 

# map ballet
mask_ballet = (data['ballet'] == 1)
# .map() for direct replacement
# ranks not in the dict will become nan by default with .map()
data.loc[mask_ballet, 'numeric_rank'] = data.loc[mask_ballet, 'rank'].map(ballet_rank_replace_dict)

# map opera ranks 
# only for rows where numeric_rank is still nan to avoid overwriting ballet if an artform could be both
mask_opera = (data['opera'] == 1) & (data['numeric_rank'].isna()) # Check if it could be both ballet and opera
data.loc[mask_opera, 'numeric_rank'] = data.loc[mask_opera, 'rank'].map(opera_rank_replace_dict)


# best rank per purchase
best_rank_agg = data.groupby(['id', 'start_date'], observed=True).agg(
    best_rank_per_purchase=('numeric_rank', 'min') # 'min' as lower number is better
).reset_index()


# merge back best rank
data = data.merge(best_rank_agg, on=['id', 'start_date'], how='left')

In [ ]:
# drop the best_rank_per_purchase na's where [data['season'].isin(['2022_2023', '2023_2024', '2024_2025', '2025_2026']
data = data[~((data['season'].isin(['2022_2023', '2023_2024', '2024_2025', '2025_2026'])) & (data['best_rank_per_purchase'].isna()))].copy()
data = data.reset_index(drop=True)

In [ ]:
data = data.drop(columns=['rank'])

In [ ]:
data = data.join(
    data.groupby(['id', 'start_date', 'ticket_type'])
    .size()
    .unstack(fill_value=0)
    .add_prefix('tickets_type_'), 
    on=['id', 'start_date']
)

In [ ]:
# drop ticket_type column
data = data.drop(columns='ticket_type')

# group by id and start_date and remove duplicates
data = data.drop_duplicates(subset=['id', 'start_date'])

In [ ]:
# remove out orders with more than 10 tickets as these are groups
data = data[data['order_size'] <= 10]

In [ ]:
data = data[data['total_order_value'] > 0]

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
# create next_purchase_date and time columns
data = data.sort_values(by=['id', 'min_purchase_date'], ascending=True)
data['next_purchase_date'] = data.groupby('id')['min_purchase_date'].shift(-1)
data = data.drop_duplicates(subset=['id'], keep='first')
data = data.reset_index(drop=True)

In [ ]:
# left censoring
# pre-attendance repurchasers
pre_attendance_repurchasers = data[
    (data['next_purchase_date'].notna()) &
    (data['next_purchase_date'] <= data['start_date'])
]

In [ ]:
data = data[~((data['next_purchase_date'].notna()) & 
                       (data['next_purchase_date'] <= data['start_date']))]

In [ ]:
# keep only records where the performance has already occurred
data = data[data['start_date'] <= retrieval_date]



In [ ]:
# add lead days feature
data['lead_days'] = (data['start_date'] - data['min_purchase_date']).dt.days

# retain only lead days that are 0 or above
data = data[data['lead_days'] >= 0]
data = data.reset_index(drop=True)


In [ ]:
# age at time of purchase
data['age_at_purchase'] = (data['min_purchase_date'] - data['birthdate']).dt.days / 365.25
data['age_at_purchase'] = data['age_at_purchase'].apply(np.floor)

# create columns for the three main categories age groups
data['age_18_35'] = 0
data['age_36_55'] = 0
data['age_over_55'] = 0

# flags for the main age groups
data.loc[(data['age_at_purchase'] >= 18) & (data['age_at_purchase'] <= 35), 'age_18_35'] = 1
data.loc[(data['age_at_purchase'] >= 36) & (data['age_at_purchase'] <= 55), 'age_36_55'] = 1
data.loc[data['age_at_purchase'] > 55, 'age_over_55'] = 1



In [ ]:
data['days_since_first_purchase'] = (retrieval_date - data['min_purchase_date']).dt.days

In [ ]:
# get month of year and day of week of start_date and min_purchase_date
data['start_month_num'] = data['start_date'].dt.month
data['start_dayofweek_num'] = data['start_date'].dt.dayofweek

data['purchase_month_num'] =  data['min_purchase_date'].dt.month

weekend_days = [4, 5, 6]
data['is_weekend_performance'] = data['start_dayofweek_num'].isin(weekend_days).astype(int)

month_map = {
    1: 'january',
    2: 'february',
    3: 'march',
    4: 'april',
    5: 'may',
    6: 'june',
    7: 'july',
    8: 'august',
    9: 'september',
    10: 'october',
    11: 'november',
    12: 'december'
}

data['start_month_name'] = data['start_month_num'].map(month_map)

data = pd.get_dummies(data, columns=['start_month_name'], prefix='start_month', dtype=int, dummy_na=False)

max_val_month = 12
data['purchase_month_sin'] = np.sin(2 * np.pi * data['purchase_month_num'] / max_val_month)
data['purchase_month_cos'] = np.cos(2 * np.pi * data['purchase_month_num'] / max_val_month)



In [ ]:
# dummies just for male and female
data['gender_male'] = (data['gender'] == 'male').astype(int)
data['gender_female'] = (data['gender'] == 'female').astype(int)

In [ ]:
# the first start_date of each production should be 1 since it is the first performance of the production, indicating premiere status
# flirt should remain 0 
data['is_premiere'] = data.groupby('production')['start_date'].transform('min')
data['is_premiere'] = (data['start_date'] == data['is_premiere']).astype(int)
data.loc[data['production'].str.contains(' flirt', case=False, na=False), 'is_premiere'] = 0

data['is_flirt'] = data['production'].str.contains(' flirt', case=False, na=False).astype(int)

# remove ' flirt' from production name
data['production'] = data['production'].str.replace(' flirt', '', case=False)

### Get the location from cities, municipalities, and then countries (in that order)

In [ ]:
from survival.utils import geonames_cleaner
geonames_cleaner(data, ['city', 'municipality'])
geonames_cleaner(nl, ['name'])
geonames_cleaner(hh, ['municipality'])
geonames_cleaner(gemeenten, ['oude_naam', 'nieuwe_naam'])
geonames_cleaner(cities, ['name'])

In [ ]:
# known dutch locations
known_nl_cities = set(nl['name'].dropna().unique()) \
    .union(set(cities.loc[cities['country_code'] == 'nl', 'name'].dropna().unique())) \
    .union(set(data.loc[data['country_code'] == 'nl', 'city'].dropna().unique()))
known_nl_municipalities = set(hh['municipality'].dropna().unique()) \
    .union(set(gemeenten['nieuwe_naam'].dropna().unique())) \
    .union(set(data.loc[data['country_code'] == 'nl', 'municipality'].dropna().unique()))

# nan country_codes if city or municipality is known dutch
nan_cc_city_nl_mask = data['country_code'].isna() & data['city'].isin(known_nl_cities)
data.loc[nan_cc_city_nl_mask, 'country_code'] = 'nl'
nan_cc_municipality_nl_mask = data['country_code'].isna() & data['municipality'].isin(known_nl_municipalities)
data.loc[nan_cc_municipality_nl_mask, 'country_code'] = 'nl'

# override existing non-'nl' country_codes
city_is_nl = data['city'].isin(known_nl_cities)
municipality_is_nl = data['municipality'].isin(known_nl_municipalities)
city_is_nan = data['city'].isna()
municipality_is_nan = data['municipality'].isna()

override_cond = (city_is_nl & municipality_is_nl) | \
                (city_is_nl & municipality_is_nan) | \
                (city_is_nan & municipality_is_nl)

actual_override_mask = (data['country_code'].notna() & (data['country_code'] != 'nl')) & override_cond
data.loc[actual_override_mask, 'country_code'] = 'nl'

In [ ]:
data['is_nl'] = (data['country_code'] == 'nl').astype(int)

### Postal codes longitude and latitude

In [ ]:
# for data['postcode'], remove all non-numeric characters
data['postalcode'] = data['postalcode'].astype(str).str.replace(r'[^0-9]', '', regex=True)


In [ ]:
# retain only a select amount digits of the postalcode
data['pc_4'] = data['postalcode'].str[:4]
data['pc_3'] = data['postalcode'].str[:3]
data['pc_2'] = data['postalcode'].str[:2]

In [ ]:

# lookup table
lookup_table = postalcode[['pc_4', 'latitude', 'longitude']].copy()
lookup_table.rename(columns={'latitude': 'latitude_pc_ref', 'longitude': 'longitude_pc_ref'}, inplace=True) # Rename to avoid clash
lookup_table['pc_4'] = lookup_table['pc_4'].astype(str)
lookup_table.drop_duplicates(subset=['pc_4'], keep='first', inplace=True)

# 'pc_4' is string
data['pc_4'] = data['pc_4'].astype(str)

# preserve lat/lon if they exist, before merge
if 'latitude' in data.columns:
    data.rename(columns={'latitude': 'latitude_original', 'longitude': 'longitude_original'}, inplace=True)

# merge to get PC4 coordinates
data = pd.merge(
    data,
    lookup_table,
    on='pc_4',
    how='left'
)

# final 'latitude' and 'longitude' columns with hierarchy
# Initialize with NaNs
data['latitude'] = np.nan
data['longitude'] = np.nan

# condition for using PC4 derived coordinates (dutch and successfully looked up)
dutch_pc4_success_mask = (data['country_code'] == 'nl') & data['latitude_pc_ref'].notna()

# populate with PC4 derived coordinates
data.loc[dutch_pc4_success_mask, 'latitude'] = data.loc[dutch_pc4_success_mask, 'latitude_pc_ref']
data.loc[dutch_pc4_success_mask, 'longitude'] = data.loc[dutch_pc4_success_mask, 'longitude_pc_ref']

# fallback to original coordinates if PC4-derived are nan AND original exist
if 'latitude_original' in data.columns:
    fallback_mask = data['latitude'].isna() & data['latitude_original'].notna()
    data.loc[fallback_mask, 'latitude'] = data.loc[fallback_mask, 'latitude_original']
    data.loc[fallback_mask, 'longitude'] = data.loc[fallback_mask, 'longitude_original']

# drop intermediate/original columns
cols_to_drop = ['latitude_pc_ref', 'longitude_pc_ref']
if 'latitude_original' in data.columns:
    cols_to_drop.extend(['latitude_original', 'longitude_original'])
data.drop(columns=cols_to_drop, errors='ignore', inplace=True)


### Cities longitude and latitude

In [ ]:
# city lookup table from 'nl' dataset
# cityname as key
city_lookup = nl[['name', 'latitude', 'longitude']].copy()
city_lookup.rename(columns={'latitude': 'city_lat_ref', 'longitude': 'city_lon_ref'}, inplace=True)
city_lookup.drop_duplicates(subset=['name'], keep='first', inplace=True) 

# merge city coordinates into 'data' (temporary suffixed cols)
# dont overwrite existing lat/lon
data = pd.merge(
    data,
    city_lookup,
    left_on='city',
    right_on='name',
    how='left',
    suffixes=('', '_city_lookup_temp')
).drop(columns=['name'], errors='ignore')

# names of newly merged columns
lat_col_from_city_lookup = 'city_lat_ref_city_lookup_temp' if 'city_lat_ref_city_lookup_temp' in data.columns else 'city_lat_ref'
lon_col_from_city_lookup = 'city_lon_ref_city_lookup_temp' if 'city_lon_ref_city_lookup_temp' in data.columns else 'city_lon_ref'


# conditionally fill 'latitude'/'longitude' if they are still nan
#    AND the record is Dutch AND city lookup was successful
fill_mask = (
    data['latitude'].isna() &                 
    (data['country_code'] == 'nl') &          
    data[lat_col_from_city_lookup].notna() 
)

data.loc[fill_mask, 'latitude'] = data.loc[fill_mask, lat_col_from_city_lookup]
data.loc[fill_mask, 'longitude'] = data.loc[fill_mask, lon_col_from_city_lookup]

# remove temporary merged columns
cols_to_drop_temp = [lat_col_from_city_lookup, lon_col_from_city_lookup]
data.drop(columns=cols_to_drop_temp, errors='ignore', inplace=True)

In [ ]:
alt_names_dict = {}
for _, row in nl.iterrows(): # nl should ideally have unique cleaned 'name'
    if isinstance(row['alternatenames'], str):
        main_name = row['name']
        for alt_name_raw in row['alternatenames'].split(','):
            cleaned_alt = alt_name_raw.strip() 
            if cleaned_alt:
                alt_names_dict[cleaned_alt] = main_name

# identify dutch cities in data still needing coordinates
missing_coords_mask = (data['country_code'] == 'nl') & data['latitude'].isna() & data['city'].notna()
cities_to_fill = data.loc[missing_coords_mask, 'city'].unique()

# fill coordinates using the alt_names_dict and city_lookup
for city_in_data in cities_to_fill:
    # cleaned_city_for_lookup = geonames_cleaner_function(city_in_data) # If data['city'] needs cleaning to match dict keys
    cleaned_city_for_lookup = city_in_data

    if cleaned_city_for_lookup in alt_names_dict:
        main_name = alt_names_dict[cleaned_city_for_lookup]
        coords_row = city_lookup[city_lookup['name'] == main_name]

        if not coords_row.empty:
            update_mask = (data['country_code'] == 'nl') & \
                          (data['city'] == city_in_data) & \
                          data['latitude'].isna()
            data.loc[update_mask, 'latitude'] = coords_row['city_lat_ref'].iloc[0]
            data.loc[update_mask, 'longitude'] = coords_row['city_lon_ref'].iloc[0]

In [ ]:
# check remaining missing cities
still_missing = data[
    (data['country_code'] == 'nl') & 
    (data['longitude'].isna()) & 
    (data['city'].notna())
]['city'].unique()

print(f"num of cities still missing: {len(still_missing)}")
print("still missing cities:", still_missing)

### Municipalities

In [ ]:
# dutch municipality coordinate lookup from 'nl'
nl_municipalities_src = nl[
    (nl['alternatenames'].str.contains(' munici|gemeente', case=False, na=False)) |
    (nl['name'].str.contains('gemeente', case=False, na=False)) |
    (nl['feature_code'] == 'adm2')
].copy() # use .copy()

nl_municipalities_src['name'] = nl_municipalities_src['name'].str.replace('^gemeente ', '', regex=True).str.strip()
nl_municipalities_src.loc[nl_municipalities_src['name'] == 'westvoorne', 'name'] = 'voorne aan zee'

municipality_lookup = nl_municipalities_src.sort_values('feature_code') \
    .drop_duplicates(subset=['name'], keep='first') \
    [['name', 'latitude', 'longitude']] \
    .rename(columns={'latitude': 'mun_lat_ref', 'longitude': 'mun_lon_ref'})

# merge municipality coordinates (into temporary columns)
data = pd.merge(
    data,
    municipality_lookup,
    left_on='municipality',
    right_on='name',
    how='left'
).drop(columns=['name'], errors='ignore')

# conditionally fill main 'latitude'/'longitude'
fill_mask = (data['latitude'].isna()) & \
            (data['country_code'] == 'nl') & \
            data['mun_lat_ref'].notna()

data.loc[fill_mask, 'latitude'] = data.loc[fill_mask, 'mun_lat_ref']
data.loc[fill_mask, 'longitude'] = data.loc[fill_mask, 'mun_lon_ref']

# remove temporary columns
data.drop(columns=['mun_lat_ref', 'mun_lon_ref'], errors='ignore', inplace=True)

In [ ]:
# remaining missing coordinates
still_missing = data[
    (data['longitude'].isna()) & 
    (data['city'].notna()) &
    (data['country_code'] == 'nl')
]
print(f"Number of records still missing coordinates: {len(still_missing)}")

### Countries

In [ ]:
pcli = countries[countries['feature_code'] == 'pcli'].drop(columns=['name', 'feature_code'])

In [ ]:
# mask records that need country coordinates
missing_coords_mask = (data['longitude'].isna()) & (data['country_code'] != 'nl')

# country coordinates mapping
country_coords = pcli.set_index('country_code')[['latitude', 'longitude']]

# update latitude and longitude
data.loc[missing_coords_mask, 'latitude'] = (
    data.loc[missing_coords_mask, 'country_code']
    .map(country_coords['latitude'])
)
data.loc[missing_coords_mask, 'longitude'] = (
    data.loc[missing_coords_mask, 'country_code']
    .map(country_coords['longitude'])
)

In [ ]:
# check the missing countries by checking the feature_code starting with 'pcl' in countries and add long and lat to data
missing_cc = data.loc[(data['longitude'].isna()) & (data['country_code'] != 'nl'), 'country_code'].unique()

missing_countries = countries[
	countries['country_code'].isin(missing_cc) & 
	countries['feature_code'].str.startswith('pcl', na=False)
].drop_duplicates(subset='country_code')

data.loc[
	(data['longitude'].isna()) & (data['country_code'].notna()), 
	'longitude'
] = data.loc[
	(data['longitude'].isna()) & (data['country_code'].notna()), 
	'country_code'
].map(missing_countries.set_index('country_code')['longitude'])

data.loc[
	(data['latitude'].isna()) & (data['country_code'].notna()), 
	'latitude'
] = data.loc[
	(data['latitude'].isna()) & (data['country_code'].notna()), 
	'country_code'
].map(missing_countries.set_index('country_code')['latitude'])

In [ ]:
# manually look up the lat and long of american samoa and namibia
american_samoa = (-14.23377, -169.47767)
namibia = (-22.00000, 17.00000)

# update the lat and long of american samoa and namibia
data.loc[data['country_code'] == 'as', 'latitude'] = american_samoa[0]
data.loc[data['country_code'] == 'as', 'longitude'] = american_samoa[1]

data.loc[data['country_code'] == 'na', 'latitude'] = namibia[0]
data.loc[data['country_code'] == 'na', 'longitude'] = namibia[1]


### Get the distance of known longitude and latitude to nob

In [ ]:
# for every lan and long, calculate the distance to the nob
nob = (52.367608492466346, 4.901889580039182)

data['distance_to_nob'] = data.apply(
    lambda row: geodesic(nob, (row['latitude'], row['longitude'])).kilometers
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']) else None,
    axis=1
)


### Median Income

In [ ]:
# get the income from hh and match with data municipality
data = data.merge(hh, left_on='municipality', right_on='municipality', how='left')

### Split

In [ ]:
# training set for imputation is a 365 days minus the max purchase date
cutoff = retrieval_date - pd.Timedelta(days=365)

# ensure all date columns are datetime type
data['start_date'] = pd.to_datetime(data['start_date'])
data['purchase_date'] = pd.to_datetime(data['purchase_date'])
data['next_purchase_date'] = pd.to_datetime(data['next_purchase_date'])

# extract just the date part of start_date for calculations
start_date_only = data['start_date'].dt.date

# calculate the end of observation window from performance date
observation_end = pd.to_datetime(start_date_only) + pd.Timedelta(days=365)

# for train, cap observation window at cutoff date
training_mask = data['start_date'] < cutoff
observation_end[training_mask] = observation_end[training_mask].clip(upper=cutoff)

# for test, cap observation window at retrieval_date
test_mask = data['start_date'] >= cutoff
observation_end[test_mask] = observation_end[test_mask].clip(upper=retrieval_date)

# event column
data['event'] = 0

# if next purchase is within observation window
has_next_purchase = data['next_purchase_date'].notna()
within_window = has_next_purchase & (data['next_purchase_date'] <= observation_end)
data.loc[within_window, 'event'] = 1

# calculate time using date-only version of start_date
# for events: time from performance date to next purchase
mask_event = data['event'] == 1
data.loc[mask_event, 'time'] = (
    data.loc[mask_event, 'next_purchase_date'] - pd.to_datetime(start_date_only[mask_event])
).dt.days

# for censored: time from performance date to end of observation window
mask_censored = data['event'] == 0
data.loc[mask_censored, 'time'] = (
    observation_end[mask_censored] - pd.to_datetime(start_date_only[mask_censored])
).dt.days

# ensure time doesnt exceed 365 days and is not negative
data['time'] = data['time'].clip(lower=0, upper=365)

data = data.reset_index(drop=True)

### Imputation Split

In [ ]:
impute_train_set = data[data['start_date'] < cutoff]

In [ ]:
# create binary flag to indicate whether distance to nob is known
data['distance_imputed'] = data['distance_to_nob'].isna().astype(int) 

# fill missing distances with the median of the distance of country_code nl of the impute set
data['distance_to_nob'] = data['distance_to_nob'].fillna(impute_train_set[impute_train_set['country_code'] == 'nl']['distance_to_nob'].median())

In [ ]:
print("\nBEFORE IMPUTATION:")
train_missing_pct = impute_train_set['median_income'].isna().mean() * 100
test_missing_pct = data[data['start_date'] >= cutoff]['median_income'].isna().mean() * 100
print(f"- Train data missing income: {train_missing_pct:.2f}%")
print(f"- Test data missing income: {test_missing_pct:.2f}%")


In [ ]:
train_muni = impute_train_set['city'].notna().mean() * 100
test_muni = data[data['start_date'] >= cutoff]['city'].notna().mean() * 100
print(f"- train data with city: {train_muni:.2f}%")
print(f"- test data with city: {test_muni:.2f}%")

In [ ]:
# initialize imputation flags

data['foreign_income_imputed'] = 0

# median values for fallback options using train only
nl_median = impute_train_set.loc[impute_train_set['country_code'] == 'nl', 'median_income'].median()
foreign_median = impute_train_set.loc[(impute_train_set['country_code'] != 'nl') & 
                                    impute_train_set['country_code'].notna(), 'median_income'].median()

# country income lookup dictionary
country_income_map = income.set_index('country_code')['median_income'].to_dict()

# imputation for Dutch records with coordinates
# create and train the imputer using train
nl_train_data = impute_train_set[
    (impute_train_set['country_code'] == 'nl') & 
    impute_train_set['latitude'].notna() & 
    impute_train_set['longitude'].notna() &
    impute_train_set['median_income'].notna()  # only known values
]

# features and target
features = ['latitude', 'longitude', 'distance_to_nob']
target = 'median_income'

# fit the imputer (only if theres train data)
if not nl_train_data.empty:
    imputer = IterativeImputer(
        estimator=RandomForestRegressor(
            n_estimators=50, 
            random_state=42,
            max_depth=10
        ),
        random_state=42,
        max_iter=10,
        initial_strategy='mean'
    )
    
    # fit
    imputer.fit(nl_train_data[features + [target]])
    
    # apply to all data that needs imputation
    nl_missing = data[
        (data['country_code'] == 'nl') & 
        data['latitude'].notna() & 
        data['longitude'].notna() &
        data['median_income'].isna()
    ]
    
    if not nl_missing.empty:
        missing_indices = nl_missing.index
        X_missing = nl_missing[features + [target]].copy()
        
        # transform using the trained imputer
        X_imputed = imputer.transform(X_missing)
        
        # update values
        data.loc[missing_indices, 'median_income'] = X_imputed.values[:, -1]




# country-specific lookup for non-dutch records
non_nl_mask = (
    data['country_code'].notna() & 
    (data['country_code'] != 'nl') & 
    data['median_income'].isna()
)

# original state to check which ones get filled
original_non_nl_missing = data.loc[non_nl_mask, 'median_income'].copy()

# apply country-specific values
data.loc[non_nl_mask, 'median_income'] = data.loc[non_nl_mask, 'country_code'].map(country_income_map)

# flag successfully imputed values
actually_imputed_mask = non_nl_mask & data['median_income'].notna() & original_non_nl_missing.isna()
data.loc[actually_imputed_mask, 'foreign_income_imputed'] = 1

# dutch median for remaining dutch records
remaining_nl_mask = (data['country_code'] == 'nl') & data['median_income'].isna()
if pd.notna(nl_median) and remaining_nl_mask.any():
    data.loc[remaining_nl_mask, 'median_income'] = nl_median


# dutch median for records with unknown country code
unknown_country_mask = data['country_code'].isna() & data['median_income'].isna()
if pd.notna(nl_median) and unknown_country_mask.any():
    data.loc[unknown_country_mask, 'median_income'] = nl_median


# fill remaining missing values with dutch median as final fallback
still_missing_mask = data['median_income'].isna()
if pd.notna(nl_median) and still_missing_mask.any():
    data.loc[still_missing_mask, 'median_income'] = nl_median

print(f"foreign records imputed: {data['foreign_income_imputed'].sum()}")
print(f"records still missing income: {data['median_income'].isna().sum()}")

In [ ]:
# # After running validation
# results_df = validate_imputation(nl_data, features, target)

# # Calculate confidence intervals
# for metric in ['RMSE', 'MAE', 'R2', 'NRMSE']:
#    mean = results_df[metric].mean()
#    std = results_df[metric].std()
#    ci = std * 1.96  # 95% ci
#    print(f"\n{metric}:")
#    print(f"Mean: {mean:.4f} ± {ci:.4f}")

In [ ]:
# process postal code information for all records
data['pc_4'] = data['pc_4'].astype(str)
data['pc_3'] = data['pc_4'].str[:3]  
data['pc_2'] = data['pc_4'].str[:2] 

# valid records for detailed location binning
valid_records = (
    (data['country_code'] == 'nl') & 
    (data['distance_imputed'] == 0) & 
    (data['pc_4'].str.len() == 4) & 
    (data['pc_4'].str.isdigit()) &
    (~data['pc_4'].str.startswith('0'))
)

# location_bin column with default value
data['location_bin'] = 'Other'  # default

# apply specific location binning for valid dutch records
# amsterdam postal codes mpre granular
def apply_location_bin(row):
    try:
        pc3 = int(row['pc_3'])
        # amsterdam postal codes
        if 100 <= pc3 <= 111:
            return f"AMS_PC3_{row['pc_3']}"
        else:
            # other Dutch postal codes with valid format
            return f"NL_PC2_{row['pc_2']}"
    except (ValueError, TypeError):
        return 'Invalid_PC4'  # For any conversion errors

# process records that meet criteria
data.loc[valid_records, 'location_bin'] = data.loc[valid_records].apply(apply_location_bin, axis=1)

# data for target encoding
training_data = data[data['start_date'] < cutoff].copy()
training_data = training_data.dropna(subset=['location_bin', 'event'])

# fit the target encoder
target_encoder = TargetEncoder(
    cols=['location_bin'],
    handle_unknown='value',
    handle_missing='value',
    smoothing=20.0
)

target_encoder.fit(training_data[['location_bin']], training_data['event'])

# apply encoding
data['location_bin_encoded_rate'] = target_encoder.transform(data[['location_bin']])

In [ ]:
# does not exist post-covid
data = data.drop(columns=['tickets_type_migration'])

In [ ]:
# opera 1, ballet 0
data = data.rename(columns={
    'opera': 'artform'}).drop(columns=['ballet'])

In [ ]:
# relevant seasons
data = data.loc[data['season'].isin(['2022_2023', '2023_2024', '2024_2025', '2025_2026'])].copy()
data = data.reset_index(drop=True)

In [ ]:
standard_features = [
    # Order metrics
    'order_size', 
    'total_order_value',
    #'avg_order_value',
    'artform',
    'best_rank_per_purchase',
    'lead_days',
    
    # performance metrics
    'perf_sales_cumulative', # maybe remove
    'prod_sales_cumulative', 
    'daily_perf_sales', 
    'daily_prod_sales', 
    #'final_perf_sales', 
    'perf_sales_percentage',
    'perf_free_tickets_cumulative', 
    'perf_operational_discount_cumulative',
    
    # timing features
    'days_since_sale_start', 'sale_period_duration_days', 'days_left_in_season',
    
    # audience type and occupancy
    'subscriber_percentage', 'regular_percentage', 'occupancy_rate',
    
    # ticket types
    'tickets_type_artform_discount', 'tickets_type_regular',
    'tickets_type_special_discount', 'tickets_type_stadspas', 'tickets_type_student',
    'tickets_type_youth_16', 'tickets_type_youth_35',
    
    # customer demographics
    'age_18_35', 'age_36_55', 'age_over_55', 'gender_male', 'gender_female', 'is_nl',
    
    # performance attributes
    'is_weekend_performance', 'is_premiere', 'is_flirt',
    
    # seasonality 
    'start_month_april', 'start_month_december', 'start_month_february',
    'start_month_january', 'start_month_july', 'start_month_june',
    'start_month_march', 'start_month_may', 'start_month_november',
    'start_month_october', 'start_month_september',
    'purchase_month_sin', 'purchase_month_cos',
    
    # location data
    'distance_to_nob', 'median_income', 'distance_imputed',
    'foreign_income_imputed', 'location_bin_encoded_rate',

    # time event
    'time', 'event'
]

start = ['start_date']

# scale certain features
features_to_scale = [
    'order_size', 
    'total_order_value', 
    #'avg_order_value',
    'lead_days',
    'best_rank_per_purchase', 
    'perf_sales_cumulative', 
    'prod_sales_cumulative',
    'daily_perf_sales', 
    'daily_prod_sales', 
    #'final_perf_sales',
    'perf_free_tickets_cumulative', 
    'perf_operational_discount_cumulative',
    'days_since_sale_start', 
    'sale_period_duration_days', 
    'days_left_in_season',
    'distance_to_nob', 
    'median_income', 
    'tickets_type_artform_discount', 
    'tickets_type_regular',
    'tickets_type_special_discount', 
    'tickets_type_stadspas', 
    'tickets_type_student',
    'tickets_type_youth_16', 
    'tickets_type_youth_35'
]

In [ ]:
# features for scaling
data = data[standard_features + start].copy()

# split at cutoff
train_data = data[data['start_date'] < cutoff].copy()
eval_data = data[data['start_date'] >= cutoff].copy()

# fit scaler 
scaler = StandardScaler()
scaler.fit(train_data[features_to_scale])

# transform train and eval
train_data[features_to_scale] = scaler.transform(train_data[features_to_scale])
eval_data[features_to_scale] = scaler.transform(eval_data[features_to_scale])

train_data = train_data.drop(columns=['start_date'])
eval_data = eval_data.drop(columns=['start_date'])

# recombine back to full dataset if needed
data = pd.concat([train_data, eval_data])

In [ ]:
# create hyperparameter tuning splits from train_data
test, val = train_test_split(
    eval_data, 
    test_size=0.5, 
    stratify=eval_data['event'], 
    random_state=1212
)

# reset indices
test = test.reset_index(drop=True)
val = val.reset_index(drop=True)

# remove target variables if not in the feature lists
standard_features = [f for f in standard_features if f not in ['time', 'event']]


# Define export path
export_path = '../../data/processed/'

# convert numeric to float32 
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
for df in [train_data, eval_data, test, val]:
    df[numeric_cols] = df[numeric_cols].astype('float32')

# export complete datasets
data.to_parquet(f'{export_path}complete_dataset.parquet', index=False)

# export features and targets separately
# train
train_data[standard_features + ['event', 'time']].to_parquet(f'{export_path}exploratory_data.parquet', index=False)
train_data[standard_features].to_parquet(f'{export_path}train_standard_features.parquet', index=False)
train_data[['time', 'event']].to_parquet(f'{export_path}train_targets.parquet', index=False)

# test
test[standard_features].to_parquet(f'{export_path}test_standard_features.parquet', index=False)
test[['time', 'event']].to_parquet(f'{export_path}test_targets.parquet', index=False)

# validation
val[standard_features].to_parquet(f'{export_path}val_standard_features.parquet', index=False)
val[['time', 'event']].to_parquet(f'{export_path}val_targets.parquet', index=False)

# export feature lists as JSON 
import json
with open(f'{export_path}feature_lists.json', 'w') as f:
    json.dump({
        'standard_features': standard_features,
    }, f, indent=2)

print(f"exported {len(data)} total records")
print(f"train: {len(train_data)} records")
print(f"tandard features: {len(standard_features)}")
print(f"test: {len(test)} records")
print(f"val: {len(val)} records")
